In [ ]:
import numpy as np 

import matplotlib.pyplot as plt

from delta_sigma_simulator.modulator import DeltaSigmaModulator
from delta_sigma_simulator.filter import FilterFirstOrder
from delta_sigma_simulator.quantizer import QuantizerDelayHysteresis

In [ ]:
# Quantizer delay
td = 25e-12
# Loop filter gain
a = 1
# Loop filter cut-off frequency
f = np.logspace(8, 10, 101)
# Loop filter time constant
tau = 1 / (2 * np.pi * f)
# Design parameter
mu = 2 * (np.exp(td / tau) - 1)
# Predicted self-oscillation frequency
f0 = 1 / (2 * tau) * 1 / np.log(1 + mu)
# Predicted self-oscillation frequency by A. Babaie-Fishani
f0_old = 1 / (4 * td) * (1 + td / tau) / (1 + 1 / 2 * td / tau)
# Simulated self-oscillation frequency
f0_sim = np.zeros_like(f)

for i, fi in enumerate(f):
    # Loop filter
    loop_filter = FilterFirstOrder(fi, a)
    # Quantizer
    quantizer = QuantizerDelayHysteresis(td, 0.0)
    # SDM
    asdm = DeltaSigmaModulator(loop_filter, quantizer)

    y, v, e = asdm.simulate(0, 0, np.array([0, 10 / f0[i]]))

    assert np.isclose(e[-1] - e[-3], e[-3] - e[-5]), "Simulation failed to converge."
        
    f0_sim[i] = 1 / (e[-1] - e[-3])

In [ ]:
plt.loglog(f, np.abs(f0 - f0_sim), 'k', label="This work")
plt.loglog(f, np.abs(f0_old - f0_sim), 'k--', label="Prior work")
plt.xlabel(r"$\frac{1}{2\pi\tau}$ [Hz]")
plt.ylabel(r"$|f_0-f_{0,sim}|$ [Hz]")
plt.xlim(1e8, 1e10)
plt.ylim(1e-6, 1e9)
plt.grid(which='both', linestyle='--')
plt.legend()
plt.show()

In [ ]:
np.savetxt("f-tau.csv", np.stack([f, np.abs(f0 - f0_sim), np.abs(f0_old - f0_sim)], axis=1), delimiter=",", header="f,abs_f0_diff,abs_f0_old_diff", comments="")

In [ ]:
# Input signal 
u = np.linspace(-1 + 1e-5, 1 - 1e-5, 101)
# Quantizer delay
td = 25e-12
# Loop filter gain
a = 1
# Loop filter cut-off frequency
f = 10e9 
# Loop filter time constant
tau = 1 / (2 * np.pi * f)
# Design parameter
mu = 2 * (np.exp(td / tau) - 1)
# Predicted self-oscillation frequency
f0 = 1 / (2 * tau) * 1 / np.log(1 + mu)
# Simulated self-oscillation frequency
f0_sim = np.zeros_like(u)
# Simulated output voltage
v_sim = np.zeros_like(u)

for i, ui in enumerate(u):
    # Loop filter
    loop_filter = FilterFirstOrder(f, a)
    # Quantizer
    quantizer = QuantizerDelayHysteresis(td, 0.0)
    # SDM
    asdm = DeltaSigmaModulator(loop_filter, quantizer)

    y, v, e = asdm.simulate(ui, 0, np.array([0, 10 / f0]))

    assert np.isclose(e[-1] - e[-3], e[-3] - e[-5]), "Simulation failed to converge."
        
    f0_sim[i] = 1 / (e[-1] - e[-3])

    v_sim[i] = v[-1] * (-1 + 2 * (e[-2] - e[-3]) / (e[-1] - e[-3]))

# Update predicted self-oscillation frequency
f0 = 1 / (2 * tau) * 1 / np.log(1 + mu) - 1 / (2 * tau) * (1 / 2 + 1 / mu) * v_sim ** 2
# Update predicted self-oscillation frequency by A. Babaie-Fishani
f0_old = 1 / (4 * td) * (1 + td / tau) / (1 + 1 / 2 * td / tau) * (1 - v_sim ** 2)

In [ ]:
plt.plot(v_sim, f0_sim * 1e-9, 'ko', label="Simulated")
plt.plot(v_sim, f0 * 1e-9, 'k-', label="This work")
plt.plot(v_sim, f0_old * 1e-9, 'k--', label="Prior work")
plt.xlabel(r"$V$ []")
plt.ylabel(r"$f_0$ [GHz]")
plt.grid(which='both', linestyle='--')
plt.legend()
plt.show()

In [ ]:
plt.semilogy(v_sim, np.abs(f0 - f0_sim), 'k-', label="This work")
plt.semilogy(v_sim, np.abs(f0_old - f0_sim), 'k--', label="Prior work")
plt.xlabel(r"$V$ []")
plt.ylabel(r"$|f_0 - f_{0,sim}|$ [Hz]")
plt.grid(which='both', linestyle='--')
plt.legend()
plt.show()

In [ ]:
np.savetxt("f-v.csv", np.stack([v_sim, f0, f0_old, f0_sim, np.abs(f0 - f0_sim), np.abs(f0_old - f0_sim)], axis=1), delimiter=",", header="v,f0,f0_old,f0_sim,abs_f0_diff,abs_f0_old_diff", comments="")

In [ ]:
# Quantizer delay
td = 25e-12
# Loop filter gain
a = 1
# Loop filter cut-off frequency
f = np.logspace(8, 10, 101)
# Loop filter time constant
tau = 1 / (2 * np.pi * f)
# Design parameter
mu = 2 * (np.exp(td / tau) - 1)

u1 = (1 + 1 / mu) * np.log(1 + mu)
u3 = u1 * (-(1 / 3 + 1 / mu + 1 / mu ** 2) * np.log(1 + mu) ** 2 + (1 / 2 + 1 / mu) * np.log(1 + mu))

iip3 = np.sqrt(-2 * u1 ** 3 / u3)

f0_old = 1 / (4 * td) * (1 + td / tau) / (1 + 1 / 2 * td / tau)
u1_old = (1 + td / tau + td ** 2 / 2 / tau ** 2 - 1 / 24 / tau ** 2 / f0_old ** 2)
u3_old = -1 / 24 / tau ** 2 / f0_old ** 2

iip3_old = np.sqrt(-2 * u1_old ** 3 / u3_old)

u1_sim = np.zeros_like(f)
u3_sim = np.zeros_like(f)

for i, fi in enumerate(f):
    # Loop filter
    loop_filter = FilterFirstOrder(fi, a)
    # Quantizer
    quantizer = QuantizerDelayHysteresis(td, 0.0)
    # SDM
    asdm = DeltaSigmaModulator(loop_filter, quantizer)

    u = np.linspace(-0.01, 0.01, 51)
    v = np.zeros_like(u)

    for j, uj in enumerate(u):
        y, v_, e = asdm.simulate(uj, 0, np.array([0, 10 / f0[i]]))

        assert np.isclose(e[-1] - e[-3], e[-3] - e[-5]), "Simulation failed to converge."
            
        v[j] = v_[-1] * (-1 + 2 * (e[-2] - e[-3]) / (e[-1] - e[-3]))
    
    _, u1_sim[i], _, u3_sim[i] = np.polynomial.polynomial.polyfit(v, u, 3)

iip3_sim = np.sqrt(-2 * u1_sim ** 3 / u3_sim)

In [ ]:
plt.semilogx(f, u1_sim, 'ko', label="Simulated")
plt.semilogx(f, u1, 'k-', label="This work")
plt.semilogx(f, u1_old, 'k--', label="Prior work")
plt.xlabel(r"$\frac{1}{2\pi\tau}$ [Hz]")
plt.ylabel(r"$U_1$ []")
plt.xlim(1e8, 1e10)
plt.grid(which='both', linestyle='--')
plt.legend()
plt.show()

In [ ]:
plt.semilogx(f, u3_sim, 'ko', label="Simulated")
plt.semilogx(f, u3, 'k-', label="This work")
plt.semilogx(f, u3_old, 'k--', label="Prior work")
plt.xlabel(r"$\frac{1}{2\pi\tau}$ [Hz]")
plt.ylabel(r"$U_3$ []")
plt.xlim(1e8, 1e10)
plt.grid(which='both', linestyle='--')
plt.legend()
plt.show()

In [ ]:
plt.semilogx(f, 20 * np.log10(iip3_sim), 'ko', label="Simulated")
plt.semilogx(f, 20 * np.log10(iip3), 'k-', label="This work")
plt.semilogx(f, 20 * np.log10(iip3_old), 'k--', label="Prior work")
plt.xlabel(r"$\frac{1}{2\pi\tau}$ [Hz]")
plt.ylabel(r"$A_{iip3}$ [dB]")
plt.xlim(1e8, 1e10)
plt.grid(which='both', linestyle='--')
plt.legend()
plt.show()

In [ ]:
np.savetxt("f-iip3.csv", np.stack([f, u1, u1_old, u1_sim, u3, u3_old, u3_sim, iip3, iip3_old, iip3_sim], axis=1), delimiter=",", header="f,u1,u1_old,u1_sim,u3,u3_old,u3_sim,iip3,iip3_old,iip3_sim", comments="")